# The UVM Test

### The run\_phase Method

*Note: The code below needs to be run in a freshly started Jupyter notebook kernal. This means after you run it once, you must restart the kernal by pressing the >> button above will run it cleanly.

In [1]:
from pyuvm import *
import proxy_pkg
import random
import time

class AluTest(uvm_test):
    @staticmethod
    def alu_op(A, B, op, error = False):
        """Python model of the TinyALU"""
        assert isinstance(op, proxy_pkg.Ops), "The tinyalu op must be of type ops"
        if op == proxy_pkg.Ops.ADD:
            result = A + B
        elif op == proxy_pkg.Ops.AND:
            result = A & B
        elif op == proxy_pkg.Ops.XOR:
            result = A ^ B
        elif op == proxy_pkg.Ops.MUL:
            result = A * B
        time.sleep(0.1)
        if error and (random.randint(0,3) == 0):
            result = result + 1
        return result

    def run_phase(self):
        self.raise_objection()  ## You MUST raise an objection
        cvg = set() #functional coverage
        for _ in range(3):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(proxy_pkg.Ops))
            cvg.add(op)
            predicted_result = self.alu_op(A, B, op)
            actual_result = self.alu_op(A, B, op, error=True)
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")
        if len(set(proxy_pkg.Ops) - cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(proxy_pkg.Ops)-cvg}")
        self.drop_objection()  ## drop the objection to end

uvm_root().run_test("AluTest")

ERROR: <ipython-input-1-81eb5ae41787>(37)[uvm_test_top]: FAILED: 6f XOR 69 = 0007 expected 0006
INFO: <ipython-input-1-81eb5ae41787>(35)[uvm_test_top]: PASSED: 96 MUL d3 = 7ba2
INFO: <ipython-input-1-81eb5ae41787>(35)[uvm_test_top]: PASSED: 2a ADD a3 = 00cd
ERROR: <ipython-input-1-81eb5ae41787>(39)[uvm_test_top]: Functional coverage error. Missed: {<Ops.AND: 2>}


In [2]:
import proxy_pkg